## Data Exploration, Feature Engineering and Experimentation

Now we're to the fun part - the Data Science. Now that the data engineers have cleaned and loaded the data to the `trips` table, we can begin our model development. For this, we will leverage Snowpark to do the **feature preparation and exploratory analysis**.   This dataset is initially ~100 million rows and is likely too large to fit into memory on our local machine or even a reasonable sized single VM in the cloud. The Snowpark Python client-side Dataframe API 
allows us to push-down most of the computation for preparation and feature engineering to Snowpark. For security and goverance reasons we can read data into memory for model training and inference but no intermediate data products can be stored outside of Snowflake.  

Input: Data in `trips` table.   
Output: Feature engineering logic.  Train function.  Predict function.

For this demo we will rewind in time and assume that it is January 1, 2020.  With the bulk ingest we have 92M records from 2013 to January 2020.

### 1. Load the Credentials


In [ ]:
from dags.snowpark_connection import snowpark_connect
session, state_dict = snowpark_connect()

### 2. Exploratory Data Analysis

In [ ]:
import snowflake.snowpark as snp
from snowflake.snowpark import functions as F
from snowflake.snowpark import types as T

import pandas as pd
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.metrics import mean_squared_error

#import logging
#logging.basicConfig(level=logging.WARN)
#logging.getLogger().setLevel(logging.DEBUG)

In [ ]:
state_dict['trips_table_name']='TRIPS'

In [ ]:
session.table(state_dict['trips_table_name']).select(F.min('STARTTIME'), F.max('STARTTIME')).show()
session.table(state_dict['trips_table_name']).count()

This is too large to fit in memory on my local system.  Lets summarize trips to daily resolution and inspect the first ten rows

In [ ]:
snowdf = session.table(state_dict['trips_table_name'])
snowdf.with_column('DATE', F.to_date('STARTTIME')).group_by('DATE').count().sort('DATE').show(10)

Once we aggregate the data at the day level we have a small enough dataset to fit in memory.  But we may want to provide a more granular time series (ie. hour or minute-level) or perhaps our data will grow considerably over time.  In either case we can't rely on in-memory computation and will want to push-down as much computation as possible to Snowflake.  
  
For exploration purposes we can see a good daily and annual seasonality in the historical data.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

df = snowdf.with_column("date", F.to_date("STARTTIME")).group_by("date").count().sort("date").to_pandas()
df.head()

plt.figure(figsize=(15, 8))
ax = sns.lineplot(x='DATE', y='COUNT', data=df)

We may not be able to get a good model that can predict across ALL stations.  Lets start with just the busiest station(s).

In [ ]:
snowdf.filter(F.col('START_STATION_ID').is_not_null()) \
      .group_by('START_STATION_ID') \
      .count() \
      .sort('COUNT', ascending=False).show()

Initially we will build with the busiest station "Central Park S & 6 Ave" which is STATION_ID=519.  Later we will see how Snowpark Python allows us to tap into the powerful horizontally, scalable compute engine to parallelize this across all stations.

In [ ]:
top_stations = snowdf.filter(F.col('START_STATION_ID').is_not_null()) \
                                                   .groupBy('START_STATION_ID') \
                                                   .count() \
                                                   .sort('COUNT', ascending=False) \
                                                   .to_pandas()['START_STATION_ID'].values.tolist()
top_stations[0:10]

In [ ]:
df = snowdf.filter(F.col('START_STATION_ID') == top_stations[0]) \
      .withColumn('DATE', 
                  F.call_builtin('DATE_TRUNC', ('DAY', F.col('STARTTIME')))) \
      .groupBy('DATE') \
      .count() \
      .sort('DATE').to_pandas()

df.head()

In [ ]:
plt.figure(figsize=(15, 8))
ax = sns.lineplot(x='DATE', y='COUNT', data=df)

We can see what looks like daily, weekly and annual periodicity as well as a slight upward trend over years. To start building the features for a regression model we might start with lag features. We can use autocorrelation to confirm our initial guess.

In [ ]:
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
%matplotlib inline
plt.style.use('seaborn-darkgrid')
plt.rc("figure", figsize=(10, 7))
plot_acf(df['COUNT'], lags=400)
plt.xlabel('Lags', fontsize=12)
plt.ylabel('Autocorrelation', fontsize=12)
plt.title('Autocorrelation of Trip Count Seasonality', fontsize=14)
plt.show()

We can definitely see the strong annual seasonality.  Lets look closer at the daily and weekly lag.

In [ ]:
plt.rc("figure", figsize=(10, 7))
plot_acf(df['COUNT'], lags=[1, 7, 30, 60, 90, 365])
plt.xlabel('Lags', fontsize=12)
plt.ylabel('Autocorrelation', fontsize=12)
plt.title('Autocorrelation of Trip Count Seasonality', fontsize=14)
plt.show()

As suspected, lags on 1, 7, 90 and 365 days have the strongest positive correlation.

Snowpark has many [functions](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/_autosummary/snowflake.snowpark.functions.html) for things like transformation, statistical analysis, etc. We will use the `lag()` window function to generate lag features very quickly on this very large dataset.
  
NOTE: In addition to the rich set of Snowpark functions, users can also tap into the wealth of [Snowflake built-in functions](https://docs.snowflake.com/en/sql-reference/functions-all.html) using the `call_builtin()` function. 

Here we will generate our features in a function.

In [ ]:
def generate_features(snowdf):
    
    snowdf = snowdf.select(F.to_date(F.col('STARTTIME')).alias('DATE'),
                           F.col('START_STATION_ID').alias('STATION_ID'))\
                   .group_by(F.col('STATION_ID'), F.col('DATE'))\
                   .count()
    
    #Impute missing values for lag columns using mean of the previous period.
    mean_1 = round(snowdf.sort('DATE').limit(1).select(F.mean('COUNT')).collect()[0][0])
    mean_7 = round(snowdf.sort('DATE').limit(7).select(F.mean('COUNT')).collect()[0][0])
    mean_90 = round(snowdf.sort('DATE').limit(90).select(F.mean('COUNT')).collect()[0][0])
    mean_365 = round(snowdf.sort('DATE').limit(365).select(F.mean('COUNT')).collect()[0][0])

    date_win = snp.Window.order_by('DATE')

    snowdf = snowdf.with_column('LAG_1', F.lag('COUNT', offset=1, default_value=mean_1) \
                                         .over(date_win)) \
                   .with_column('LAG_7', F.lag('COUNT', offset=7, default_value=mean_7) \
                                         .over(date_win)) \
                   .with_column('LAG_90', F.lag('COUNT', offset=90, default_value=mean_90) \
                                         .over(date_win)) \
                   .with_column('LAG_365', F.lag('COUNT', offset=365, default_value=mean_365) \
                                         .over(date_win)) \
                   .na.drop()
    return snowdf

This function then becomes our feature generation logic and creates essentially a DAG of aggregations and transformations which gets executed _lazily_.

In [ ]:
train_snowdf = generate_features(snowdf.filter(F.col('START_STATION_ID') == top_stations[0]))

When we call something this `show()` ALL of the processing gets pushed down into Snowflake compute for highly optimized execution.  In the end the function provides a running list of transformations to build our feature pipeline for inference and training.

In [ ]:
train_snowdf.show()

### Baseline
We might start a baseline model with ARIMA.  In this case we get an MSE of something like 56983.  See notebook 02_Data_Science_ARIMA_Baseline.ipynb for details.

Lets try pytorch tabnet to see if we can at least get this level of predictive performance.

### 3. Model Development and Experimentation
We will use [pytorch_tabnet](https://github.com/dreamquark-ai/tabnet) from the 2019 [tabnet paper](https://arxiv.org/pdf/1908.07442.pdf) by Arik, S. O. and Pfister, T.  Tabnet is a powerful deep learning framework for attentive, interpretable learning on tabular data.  Rather than substantial focus on hyper-parameter optimization  we will start with an initial set of hyper-parameters and focus on iterating over input features for now.  
  
Rather than a random split of training/validation data we will split the training dataset using a `cutpoint`.  For example we will save the final 365 days of the dataset as validation and train with the remaining data.

In [ ]:
def train(X, y, cutpoint=365, cat_idxs=[], cat_dims=[]):    
    from pytorch_tabnet.tab_model import TabNetRegressor
    import pandas as pd

    X_valid = X[-cutpoint:]
    y_valid = y[-cutpoint:]
    X_train = X[:-cutpoint]
    y_train = y[:-cutpoint]

    max_epochs = 100
    
    batch_df = pd.DataFrame(range(2,65,2), columns=['batch_size'])
    batch_df['batch_remainder'] = len(X_train)%batch_df['batch_size']
    optimal_batch_size=int(batch_df['batch_size'].where(batch_df['batch_remainder']==batch_df['batch_remainder'].min()).max())
    
    print('Selected batch size '+str(optimal_batch_size)+' for input data size: '+str(len(X_train)))
    
    regression_model = TabNetRegressor(cat_idxs=cat_idxs, cat_dims=cat_dims)

    regression_model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        max_epochs=max_epochs,
        patience=10,
        batch_size=optimal_batch_size, 
        virtual_batch_size=optimal_batch_size/2,
        num_workers=0,
        drop_last=True)
    
    return regression_model

def predict(model, X):
    y_hat = model.predict(X).reshape(-1)
    return y_hat

def forecast(df, model, lag_values:list, steps:int):
    from datetime import timedelta
    import numpy as np
    forecast_df=df.copy()
    for i in range(steps):
        station_id = forecast_df['STATION_ID'][-1:].values[0]
        future_date = forecast_df['DATE'][-1:].values[0]+timedelta(days=1)
        lags=[forecast_df['COUNT'].shift(i-1)[-1:].values[0] for i in lag_values]
        pred=round(model.predict(np.array([lags]))[0][0])
        row=[*[station_id, future_date, pred], *lags, pred]
        forecast_df.loc[len(forecast_df)]=row
    
    return forecast_df

def plot(df, x_lab:str, y_true_lab:str, y_pred_lab:str):
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    plt.figure(figsize=(15, 8))
    df = pd.melt(df, id_vars=[x_lab], value_vars=[y_true_lab, y_pred_lab])
    ax = sns.lineplot(x=x_lab, y='value', hue='variable', data=df)

We will create a pandas dataframe using to_pandas() which will generate the features as a pyarrow dataset and efficiently read them into memory in pandas locally.  
  
Let's train our first model to get a baseline. 

In [ ]:
def train_predict(df, cat_idxs=[], cat_dims=[], lag_values=[], forecast_steps=None):
    import pandas as pd
    from sklearn.metrics import mean_squared_error

    target = ['COUNT']
    feature_columns = [feature.replace('\"', '') for feature in df.columns]
    feature_columns.remove(target[0])
    feature_columns.remove('DATE')
    feature_columns.remove('STATION_ID')

    model = train(df[feature_columns].astype(float).values, 
                  df[target].values,
                  cat_idxs=cat_idxs, 
                  cat_dims=cat_dims)
    df['Y_PRED'] = predict(model, df[feature_columns].astype(float).values).astype('int')
    
    if isinstance(forecast_steps, int):
        df = forecast(df, model, lag_values=lag_values, steps=forecast_steps)

    explain_df = pd.DataFrame(model.explain(df[feature_columns].astype(float).values)[0], 
                         columns = feature_columns).add_prefix('EXPL_').round(2)
    df = pd.concat([df.set_index('DATE').reset_index(), explain_df], axis=1)
    
    MSE = mean_squared_error(y_pred=df['Y_PRED'], y_true=df[target])
    display("Error for training dataset is: "+str(MSE))
    return df.reset_index(drop=True), model, feature_columns

In [ ]:
df = train_snowdf.sort('DATE', ascending=True).to_pandas()
df, model, feature_columns = train_predict(df, lag_values=[1,7,90,365], forecast_steps=7)

In [ ]:
df

In [ ]:
plot(df[-1095:], 'DATE', 'COUNT', 'Y_PRED')

#### What if we add US holidays as a feature?
Pandas has great support for holiday lists.  We can generate a pandas dataframe and then upload it as a temporary table to create a new feature.

In [1]:
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar
from datetime import timedelta, datetime

cal = USFederalHolidayCalendar()

#generate a feature of 20 years worth of US holiday days.
start_date = datetime.strptime('2013-01-01', '%Y-%m-%d')
end_date = start_date+timedelta(days=365*20)

holiday_df = pd.DataFrame(cal.holidays(start=start_date, end=end_date), columns=['DATE'])
holiday_df['DATE'] = holiday_df['DATE'].dt.strftime('%Y-%m-%d')

In [2]:
holiday_df

,DATE
0,2013-01-01
1,2013-01-21
2,2013-02-18
3,2013-05-27
4,2013-07-04
...,...
207,2032-09-06
208,2032-10-11
209,2032-11-11
210,2032-11-25


In [ ]:
def generate_holiday_df(session):
    from snowflake.snowpark import functions as F 
    import pandas as pd
    from pandas.tseries.holiday import USFederalHolidayCalendar
    from datetime import timedelta, datetime

    cal = USFederalHolidayCalendar()

    #generate a feature of 20 years worth of US holiday days.
    start_date = datetime.strptime('2013-01-01', '%Y-%m-%d')
    end_date = start_date+timedelta(days=365*20)

    holiday_df = pd.DataFrame(cal.holidays(start=start_date, end=end_date), columns=['DATE'])
    holiday_df['DATE'] = holiday_df['DATE'].dt.strftime('%Y-%m-%d')

    session.create_dataframe(holiday_df) \
           .with_column("HOLIDAY", F.lit(1))\
           .write\
           .save_as_table("HOLIDAYS", mode="overwrite", create_temp_table=True)
    
    return session.table("HOLIDAYS")

In [ ]:
holiday_df = generate_holiday_df(session)
holiday_df.show()

Here we create an in-memory instance of the holiday dataframe.  In production we probably want to materialize this feature as a table or view.  We'll see later how that is easy to do but for now we have a function to generate it and join it to our training dataframe as a one-hot feature.

Every time we join there is a possibility of losing data from our feature set if the join column doesn't cover the same dates.  So its good to check.

In [ ]:
train_snowdf = train_snowdf.join(holiday_df, 'DATE', join_type='left') \
                           .na.fill({'HOLIDAY':0}) \
                           .sort('DATE', ascending=True)
train_snowdf.show()

Now when we train we need to specify this new holiday feature as a categorical feature.  Its already encoded (by definition) so nothing to add there.

In [ ]:
df = train_snowdf.sort('DATE', ascending=True).to_pandas()
df, model, feature_columns = train_predict(df, cat_idxs=[-1], cat_dims=[2])

In [ ]:
plot(df, 'DATE', 'COUNT', 'Y_PRED')

#### Enhancing Model Accuracy With Weather Data

Let's see how we can make our model even better with weather data. Its likely that weather and the amount of precipitation on the day will be an important signal for our model. 
  
We might start by downloading weather data or setting up an API.  But this is a lot of work when we want to build inference pipelines later.

In [ ]:
# def generate_weather_df(session):
#     from snowflake.snowpark import Window
#     from snowflake.snowpark import functions as F 
#     import pandas as pd

#     tempdf = pd.read_csv('./include/weather.csv')
#     tempdf['DATE']=pd.to_datetime(tempdf['dt_iso'].str.replace(' UTC', ''), 
#                                  format='%Y-%m-%d %H:%M:%S %z', 
#                                  utc=True).dt.tz_convert('America/New_York').dt.date
#     tempdf.columns=tempdf.columns.str.upper()
        
#     session.create_dataframe(tempdf[['DATE','RAIN_1H', 'TEMP']]) \
#            .group_by('DATE').agg([F.round(F.mean('RAIN_1H'), 2).alias('PRECIP'),
#                                  F.round(F.mean('TEMP')-F.lit(273.15), 2).alias('TEMP')])\
#            .fillna({'PRECIP':0, 'TEMP':0})\
#            .write\
#            .save_as_table("WEATHER", mode="overwrite", create_temp_table=True)

#     return session.table('WEATHER')

# weather_df = generate_weather_df(session)
# weather_df.show()
# weather_df.select(F.min('DATE'), F.max('DATE')).show()

####  Use the [Snowflake Data Marketplace](https://www.snowflake.com/data-marketplace/?_sft_dataset-category=weather)  
The Snowflake marketplace allows us to very easily access weather data (among many other types of data) and join it with the trips data.  Not only is this faster and more scalable than building from a csv file or an API, it is extremely easy to setup an MLops pipeline to keep it fresh.  
  
For this demo we will use a dataset from [Weather Source](https://app.snowflake.com/marketplace/listing/GZSOZ1LLE9).  
  
Weather Source is a leading provider of global weather and climate data and our OnPoint Product Suite provides businesses with the necessary weather and climate data to quickly generate meaningful and actionable insights for a wide range of use cases across industries.

**See notebook 02_Data_Marketplace.ipynb for details on accessing this data**.

Alternatively, run the following to subscribe.

In [ ]:
weather_listing_id = state_dict['weather_listing_id']
weather_database_name = state_dict['weather_database_name']

session.sql("CREATE OR REPLACE DATABASE "+weather_database_name+" FROM SHARE "+weather_listing_id).collect()

#The default database and schema may be reset after creating a new database
session.use_database(state_dict['connection_parameters']['database'])
session.use_schema(state_dict['connection_parameters']['schema'])

In [ ]:
def generate_weather_df(session, weather_table_name):

    weather_df = session.table(weather_table_name)\
                       .filter(F.col('POSTAL_CODE') == '10007')\
                       .select(F.col('DATE_VALID_STD').alias('DATE'), 
                               F.col('TOT_PRECIPITATION_MM').alias('PRECIP'), 
                               F.round(F.col('AVG_TEMPERATURE_FEELSLIKE_2M_C'), 2).alias('TEMP'))\
                       .sort('DATE', ascending=True)
    return weather_df

In [ ]:
weather_table_name = weather_database_name+'.ONPOINT_ID.HISTORY_DAY'

weather_df = generate_weather_df(session, weather_table_name)
weather_df.show()
weather_df.select(F.min('DATE'), F.max('DATE')).show()

We add weather features by joining the weather dataframe.

In [ ]:
train_snowdf = train_snowdf.join(weather_df, 'DATE', 'inner').sort('DATE', ascending=True)
train_snowdf.show()

Again, check the date range after the join.

In [ ]:
train_snowdf.select(F.min('DATE'), F.max('DATE')).show()

In [ ]:
df = train_snowdf.sort('DATE', ascending=True).to_pandas()
df, model, feature_columns = train_predict(df, cat_idxs=[-3], cat_dims=[2])

In [ ]:
plot(df, 'DATE', 'COUNT', 'Y_PRED')

#### Lets look at the feature importance
One reason we chose tabnet for this analysis is the built-in abilities for explainability.

In [ ]:
pd.DataFrame([model.feature_importances_], columns=feature_columns)

In [ ]:
state_dict['holiday_table_name'] = 'HOLIDAYS'
state_dict['weather_database_name'] = 'WEATHER_NYC'
state_dict['weather_table_name'] = state_dict['weather_database_name']+'.ONPOINT_ID.HISTORY_DAY'

import json
with open('./include/state.json', 'w') as sdf:
    json.dump(state_dict, sdf)

session.close()

#### Lets consolidate and upate our feature functions and save them for the ML engineering team to operationalize.
Based on the feature importance above we will go with 1 and 7 day lags only.

In [ ]:
%%writefile dags/feature_engineering.py

def generate_holiday_df(session, holiday_table_name:str):
    from snowflake.snowpark import functions as F 
    import pandas as pd
    from pandas.tseries.holiday import USFederalHolidayCalendar
    from datetime import timedelta, datetime

    cal = USFederalHolidayCalendar()

    #generate a feature of 20 years worth of US holiday days.
    start_date = datetime.strptime('2013-01-01', '%Y-%m-%d')
    end_date = start_date+timedelta(days=365*20)

    holiday_df = pd.DataFrame(cal.holidays(start=start_date, end=end_date), columns=['DATE'])
    holiday_df['DATE'] = holiday_df['DATE'].dt.strftime('%Y-%m-%d')

    session.create_dataframe(holiday_df) \
           .with_column("HOLIDAY", F.lit(1))\
           .write\
           .save_as_table(holiday_table_name, mode="overwrite", create_temp_table=True)
    
    return session.table(holiday_table_name)

def generate_weather_df(session, weather_table_name):
    from snowflake.snowpark import functions as F 
    return session.table(weather_table_name)\
                  .filter(F.col('POSTAL_CODE') == '10007')\
                  .select(F.col('DATE_VALID_STD').alias('DATE'), 
                          F.col('TOT_PRECIPITATION_MM').alias('PRECIP'), 
                          F.round(F.col('AVG_TEMPERATURE_FEELSLIKE_2M_C'), 2).alias('TEMP'))\
                  .sort('DATE', ascending=True)

def generate_features(session, input_df, holiday_table_name, weather_table_name):
    import snowflake.snowpark as snp
    from snowflake.snowpark import functions as F 
    
    #start_date, end_date = input_df.select(F.min('STARTTIME'), F.max('STARTTIME')).collect()[0][0:2]
    
    #check if features are already materialized (or in a temp table)
    holiday_df = session.table(holiday_table_name)
    try: 
        _ = holiday_df.columns
    except:
        holiday_df = generate_holiday_df(session, holiday_table_name)
        
    weather_df = session.table(weather_table_name)[['DATE','TEMP']]
    try: 
        _ = weather_df.columns
    except:
        weather_df = generate_weather_df(session, weather_table_name)[['DATE','PRECIP','TEMP']]

    feature_df = input_df.select(F.to_date(F.col('STARTTIME')).alias('DATE'),
                                 F.col('START_STATION_ID').alias('STATION_ID'))\
                         .replace({'NULL': None}, subset=['STATION_ID'])\
                         .group_by(F.col('STATION_ID'), F.col('DATE'))\
                         .count()

    #Impute missing values for lag columns using mean of the previous period.
    mean_1 = round(feature_df.sort('DATE').limit(1).select(F.mean('COUNT')).collect()[0][0])
    mean_7 = round(feature_df.sort('DATE').limit(7).select(F.mean('COUNT')).collect()[0][0])
    mean_90 = round(feature_df.sort('DATE').limit(90).select(F.mean('COUNT')).collect()[0][0])
    mean_365 = round(feature_df.sort('DATE').limit(365).select(F.mean('COUNT')).collect()[0][0])

    date_win = snp.Window.order_by('DATE')

    feature_df = feature_df.with_column('LAG_1', F.lag('COUNT', offset=1, default_value=mean_1) \
                                         .over(date_win)) \
                           .with_column('LAG_7', F.lag('COUNT', offset=7, default_value=mean_7) \
                                         .over(date_win)) \
                           .with_column('LAG_90', F.lag('COUNT', offset=90, default_value=mean_90) \
                                         .over(date_win)) \
                           .with_column('LAG_365', F.lag('COUNT', offset=365, default_value=mean_365) \
                                         .over(date_win)) \
                           .join(holiday_df, 'DATE', join_type='left').na.fill({'HOLIDAY':0}) \
                           .join(weather_df, 'DATE', 'inner') \
                          .na.drop() 

    return feature_df

Finally, train and evaluate the model with the final feature set.

In [ ]:
from dags.snowpark_connection import snowpark_connect
from dags.feature_engineering import generate_features
from snowflake.snowpark import functions as F
import pandas as pd

session, state_dict = snowpark_connect()
test_station_id = '519'

snowdf = session.table(state_dict['trips_table_name']).filter(F.col('START_STATION_ID') == test_station_id)

train_snowdf = generate_features(session=session, 
                                 input_df=snowdf, 
                                 holiday_table_name=state_dict['holiday_table_name'], 
                                 weather_table_name=state_dict['weather_table_name'])

df = train_snowdf.sort('DATE', ascending=True).to_pandas()
df, model, feature_columns = train_predict(df, cat_idxs=[-3], cat_dims=[2])

In [ ]:
plot(df, 'DATE', 'COUNT', 'Y_PRED')

In [ ]:
pd.DataFrame([model.feature_importances_], columns=feature_columns)

In [ ]:
session.close()